In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

from retrieve_columns_loghi import retrieve_columns
from get_cell_coords import get_col_row, find_loghi_words, create_table

In [2]:
# Parse the XML file
tree = ET.parse('/home/roderickmajoor/Desktop/Master/Thesis/GT_data/55/page/WBMA00007000010.xml')
root = tree.getroot()

In [3]:
# Define a function to extract information from table cells
def extract_table_cell(cell):
    cell_info = {}
    cell_info['id'] = cell.get('id')
    cell_info['row'] = int(cell.get('row', 0))
    cell_info['col'] = int(cell.get('col', 0))
    cell_info['rowSpan'] = int(cell.get('rowSpan', 1))
    cell_info['colSpan'] = int(cell.get('colSpan', 1))
    cell_info['coords'] = cell.find('{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Coords').get('points')
    text_equiv = cell.findtext('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TextEquiv/{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Unicode')
    cell_info['text'] = text_equiv if text_equiv else ''
    return cell_info

# Define a function to extract information from table regions and create DataFrames for each region
def extract_table_regions(root):
    table_regions = root.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TableRegion')
    dfs = []
    for region in table_regions:
        cells = region.findall('.//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TableCell')
        cell_data = [extract_table_cell(cell) for cell in cells]
        max_row = max(cell['row'] + cell['rowSpan'] for cell in cell_data)
        max_col = max(cell['col'] + cell['colSpan'] for cell in cell_data)
        table = [[''] * max_col for _ in range(max_row)]
        for cell in cell_data:
            for i in range(cell['row'], cell['row'] + cell['rowSpan']):
                for j in range(cell['col'], cell['col'] + cell['colSpan']):
                    table[i][j] = cell['text']
        df = pd.DataFrame(table)
        dfs.append(df)
    return dfs

In [4]:
# Extract table regions and create DataFrames
dataframes = extract_table_regions(root)

# Display DataFrames
for i, df in enumerate(dataframes):
    print(f"Table Region {i+1}:")
    print(df)
    print("\n")

Table Region 1:
         0 1     2      3   4  5
0                               
1    feb 5      35   2448       
2        6     346   3366       
3       14      67   1000       
4       16     124   1800       
5    4 mrt     120   1800       
6              905   1134       
7       15     540    668   4  8
8       16      69   1225       
9       26     155    700       
10      29     344    366       
11      30     308    861       
12    Ap 6     343    600       
13             593   1799  15   
14       9     709    652  10   
15      10     952   1029  17   
16      16     948    895  10   
17      18     954   1407   5   
18      19     330   1075       
19             949   1075       
20      20     131    300  14  8
21             982   1044  15  8
22      29     173    666  13   
23   may 2     658   1065       
24       4     515   2451       
25             420   1802   5   
26       9     426   1246  17  8
27      17     690    597       
28      27     553    150  

In [ ]:
dataframes[0]

In [ ]:
dataframes[1]

In [5]:
xml_columns = '/home/roderickmajoor/Desktop/Master/Thesis/GT_data/55/page/WBMA00007000010.xml'
xml_loghi = '/home/roderickmajoor/Desktop/Master/Thesis/loghi/data/55/page/WBMA00007000010.xml'
image_path = '/home/roderickmajoor/Desktop/Master/Thesis/GT_data/55/WBMA00007000010.jpg'

In [8]:
column_coords, row_coords = get_col_row(xml_loghi, image_path)
loghi_words_dict = find_loghi_words(xml_loghi)
df_loghi = create_table(column_coords, row_coords, loghi_words_dict)

In [11]:
df_loghi[0].reset_index(drop=True)

,0,1,2,3,4,5
0,[130],[],[],[],[],[]
1,[],[1647],[],[],[],[]
2,[],"[Jan, Eijghels, Deben]",[],[],[],[]
3,[feb],"[5, aen, Paqe, Martijn]",[35],"[1, 2448]",[],[]
4,[6],"[aen, Franc, dvicq]",[346],"[1, 3366]",[],[]
5,[14],"[aen, Jage, bours]",[67],[1000],[],[]
6,[],"[1607, Lourisz, Lusse]",[124],[1800],[],[]
7,"[1, meert]","[aen, geurt, Pietersz]",[1120],[1800],[],[]
8,[],"[aende, Coupans]",[905],[1134],[],[]
9,[15],"[aende, oost, Ind, Compe, Hoorn]",[540],[668],[4],[8]


In [12]:
dataframes[0]

,0,1,2,3,4,5
0,,,,,,
1,feb 5,,35,2448,,
2,6,,346,3366,,
3,14,,67,1000,,
4,16,,124,1800,,
5,4 mrt,,120,1800,,
6,,,905,1134,,
7,15,,540,668,4,8
8,16,,69,1225,,
9,26,,155,700,,


In [15]:
df_loghi[1].reset_index(drop=True)

,6,7,8,9,10,11
0,[],[],[],[130],[],[]
1,[],[47],[],[],[],[]
2,[],"[Moet, hebb, 395, Feb, 10]",[bal],[438116],[],[]
3,[],[],[92],[1227],[],[]
4,[],"[van, Jan, Elise]",[],[],[],[]
5,[],[ginderdeurg],[52],[7592],[],[8]
6,[],"[8117, Gerard, Luysten]",[79],[1216],[10],[]
7,[9],"[van, Do, dle, histraat]",[32],[122019],[],[]
8,[th],"[van, danit, Antgers]",[],"[394, 600]",[],[]
9,[14],"[van, dla, bistraat]",[321],[20608],[],[]


In [16]:
dataframes[1]

,0,1,2,3,4,5
0,,,,4381,16,
1,,,92,1227,,
2,,,52,759,7,8
3,8,,79,1216,10,
4,9,,32,1220,19,
5,12,,394,600,,
6,14,,32,2060,8,
7,20,,67,1000,,
8,8 maart,,320,913,10,
9,13,,394,300,,
